### Introduction

<img src="../images/RNN Diagram.png">

In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1/(1 + np.exp(-np.clip(x, -500, 500)))

### Computation for RNN Units
An RNN Unit at time-step $t$ takes as input: <br/>
* a minibatch of 'words' denoted by $x^{(t)}$, of dimensions $N \times d$, and <br/>
* the 'hidden-state' vector $h^{(t-1)}$ from the previous unit, of dimensions $N \times D_h$.

**Note: $d$ and $D_h$ are hyper-parmaters, i.e. we _chose_ to represent each hidden state using a vector of length $D_h$ and we _chose_ to use 'word embedding' vectors of length $d$.**

The code below implements a single RNN Unit's computation. The output is the 'hidden-state' vector $h^{(t)}$ for this unit, of dimensions $N \times D_h$. (In this notebook, $h^{(t)}$ for time-step $t$ is always referred to as $h\_next$).

In [3]:
def rnn_step_forward(x_t, h_prev, Wh, We, b1):
    h_next = sigmoid(np.matmul(h_prev, Wh.T) + np.matmul(x_t, We.T) + b1)
    cache = h_prev, h_next, x_t
    return h_next, cache

An RNN Unit depends on the previous RNN Unit's hidden-state (this is not different from any plain feedforward network). Therefore we sequentially run the $rnn\_step\_forward$ method implemented above, for each time step.

**Note: One crucial difference from a plain feedforward network is that each RNN Unit uses the same parameters $W_h$, $W_e$, and $b_1$. This point of difference will have a significant bearing on how we backprop through an RNN.**

The code below implements the forward pass through an RNN. We are given as inputs:
* a minibatch of 'word sequences' denoted by $x$, of dimensions $N \times T \times d$, where $N$ is the numnber of minibatches and $T$ is the length of each sequence,
* an initial state vector denoted by $h^{(0)}$ of dimensions $N \times D_h$

In [4]:
def rnn_forward(T, x, h_0, Wh, We, b1):
    N, T, d = x.shape
    _, Dh = h_0.shape
    h = np.zeros((N, T, Dh))
    h_prev = h_0
    cache_dict = {}
    for t in range(T):
        h[:, t, :],  cache_step = rnn_step_forward(x[:,t,:], h_prev, Wh, We, b1) 
        h_prev = h[:, t, :]
        cache_dict.update({t : cache_step})
    return h, cache_dict

At each time-step $t$, we are given as inputs:
* the cache for this time-step saved during our forward pass - cache stores $h^{(t)}$ and $h^{(t-1)}$,
* the gradient of total loss $J$ with respect to $h^{(t)}$, denoted by $dh\_next$, of dimensions $N \times D_h$.

In [5]:
def rnn_step_backward(dh_next, cache, Wh):
    h_prev, h_next, x_t = cache
    dsigmoid = h_next*(1 - h_next)
    interim_dot_prod = dh_next*dsigmoid       # This will be used in all equations below
    dWh_step = np.matmul(interim_dot_prod.T, h_prev)
    dWe_step = np.matmul(interim_dot_prod.T, x_t)
    db1_step = np.sum(interim_dot_prod, axis=0)
    dh_prev = np.matmul(interim_dot_prod, Wh)
    return  dWh_step, dWe_step, db1_step, dh_prev 

In [6]:
def rnn_backward(dh, cache_dict, Wh, We, b1):
    N, T, Dh = dh.shape
    dWh = np.zeros_like(Wh)
    dWe = np.zeros_like(We)
    db1 = np.zeros_like(b1)
    dh_next = np.zeros((N, Dh))
    for t in range(T, 0, -1):
        dh_next += dh[:, t-1, :]
        dWh_step, dWe_step, db1_step, dh_prev = rnn_step_backward(dh_next, cache_dict[t-1], Wh)
        dh_next = dh_prev
        dWh += dWh_step
        dWe += dWe_step
        db1 += db1_step
    return dWh, dWe, db1

### Computation for the Affine Layer 
Atop each RNN Unit, sits an Affine layer which takes the vector $h^{(t)}$ as input, applies an Affine transformation, and computes the Softmax Probability. The parameters of this layer are $U \space (Dim: D_{h} \times V )$ and $b_2 \space (Dim: V \times 1)$. 

We do not have to implement a separate Affine layer for each time-step. Unlike in the case of an RNN Unit where the computation inside it depended on the output of its previous unit, the Affine computations at each time-step are independent of each other. Therefore, once we have computed $h^{(t)}$ for each time-step, we will perform the Affine computation for ALL $T$ time-steps in one go, taking into impact the contribution from ALL mini-batches.

In [7]:
def affine_forward(h, U, b2):
    N, T, Dh = h.shape
    V = b2.shape[0]
    theta = (np.matmul(h.reshape(N*T, Dh), U.T) + b2).reshape(N, T, V)
    cache = U, b2, h
    return theta, cache

In [8]:
def affine_backward(dtheta, cache):
    U, b2, h = cache
    Dh = U.shape[1]
    N, T, V = dtheta.shape
    dh = np.matmul(dtheta.reshape(N*T, V), U).reshape(N, T, Dh)
    dU = np.matmul((dtheta.reshape(N*T, V).T), h.reshape(N*T, Dh))
    db2 = dtheta.sum(axis=(0,1))
    return dh, dU, db2

### Computation for the Softmax Layer
We will compute probabilities for all $T$ sequences in a minibatch, over all $N$ minibatches, in one go. This follows from similar reasoning as I described for the Affine layer above.

**Inputs:**
* Matrix $\theta$ of dimensions $N \times T \times V$ which stores the output of Affine Layers, and
* Matrix $y$ of dimensions $N \times T$ which stores the index in Vocabulary of the true 'word' for each time-step, for each minibatch.

**Outputs:**
* Loss over all minibatches (a single floating point number), and
* Matrix $dtheta$ of same dimensions as $\theta$, and which stores gradients of Loss w.r.t $\theta$.

**Notes:** 
* I have directly lifted the $softmax\_loss$ function from the starter code of Assignment 3 of Stanford's CS231n's [Winter 2016 edition](http://cs231n.stanford.edu/2016/).
* This version of $softmax\_loss$ uses a 'mask', an array of dimensions $N \times T$ which indicates which time-steps in a minibatch should not be counted towards the loss. This is used to handle sequences whose length is less than $T$ - we pad them with zeros (in my implementation) to increase their length to $T$, which makes for easy code elsewhere. 
* I have previously discussed maths for backprop through a Softmax layer in a [blog post](https://talwarabhimanyu.github.io/blog/2017/05/20/softmax-backprop). You may refer to $Equation \space 1.3$ in that post which derives the gradient of Loss w.r.t $\theta$.

In [156]:
def softmax(theta, y, mask):
    N, T, V = theta.shape
    theta_flat = theta.reshape(N*T, V)
    y_flat = y.reshape(N*T)
    mask_flat = mask.reshape(N*T)
    
    probs = np.exp(theta_flat - np.max(theta_flat, axis=1, keepdims=True))
    probs /= np.sum(probs, axis=1, keepdims=True)
    loss = -np.sum(mask_flat * np.log(probs[np.arange(N * T), y_flat])) / N
    dtheta_flat = probs.copy()
    dtheta_flat[np.arange(N * T), y_flat] -= 1
    dtheta_flat /= N
    dtheta_flat *= mask_flat[:, None]
    
    dtheta = dtheta_flat.reshape(N, T, V)
    return loss, dtheta

### Numerical Evaluation of Gradients to Check Correctness of our Implementation
If you are familiar with how to numerically check gradients for a network, you can skip this section and move on to [Training our RNN](#sec_id)

Below, the function $eval\_grad$ evaluates the gradient of a given function $f$ at a point $x$. This point $x$ can be multidimensional, for example I will use the $2D$ matrix $W_h$ as a 'point'. The 'gradient' is basically the change in Loss due to an infinitesimally small perturbation to the point $x$.

Notice in the code below that I have multiplied by $dh$ to calculate the gradient. This is because we are going to be passing $rnn\_forward$ and $rnn\_step\_forward$ for the argument $f$. Both these functions return the vector $h$ and not the scalar Loss which we need to compute the gradient w.r.t point $x$. Therefore we need to multiple by $dh$ to get our gradient, which is what we pass for the argument $df$.

**Note: Below, I have used the numerical gradient evaluation functions provided for assignments of Stanford's course CS321n, _"Convolutional Neural Networks for Visual Recognition"_. Specifically, I have used code from the [Winter 2016 edition](http://cs231n.stanford.edu/2016/).  **

In [10]:
def eval_grad(f, x, df):
    grad = np.zeros_like(x)
    epsilon = 1e-5
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        orig_val = x[idx]
        x[idx] = orig_val + epsilon
        fwd_fx = f(x)
        x[idx] = orig_val - epsilon
        bck_fx = f(x)
        grad[idx] = np.sum((fwd_fx - bck_fx)*df/(epsilon*2))
        x[idx] = orig_val
        it.iternext()
    return grad
def rel_error(x, y):
    # Returns relative error
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [11]:
""" Check Gradients for single RNN Unit """

np.random.seed(10151)
N, Dh, d, V = 2, 5, 3, 50
# Parameters
Wh = np.random.randn(Dh, Dh)
b1 = np.random.randn(Dh)
We = np.random.randn(Dh, d)

# Inputs
x_t = np.random.randn(N, d)
h_prev = np.random.randn(N, Dh)

# Test functions
fWh = lambda Wh: rnn_step_forward(x_t, h_prev, Wh, We, b1)[0]
fWe = lambda We: rnn_step_forward(x_t, h_prev, Wh, We, b1)[0]
fb1 = lambda b1: rnn_step_forward(x_t, h_prev, Wh, We, b1)[0]
fh_prev = lambda h_prev: rnn_step_forward(x_t, h_prev, Wh, We, b1)[0]

# Evaluate test functions
h_next, cache_step = rnn_step_forward(x_t, h_prev, Wh, We, b1)
dh_next = np.random.randn(*h_next.shape)
dWh, dWe, db1, dh_prev = rnn_step_backward(dh_next, cache_step, Wh)
dWh_num = eval_grad(fWh, Wh, dh_next)
dWe_num = eval_grad(fWe, We, dh_next)
db1_num = eval_grad(fb1, b1, dh_next)
dh_prev_num = eval_grad(fh_prev, h_prev, dh_next)
print('Error in dWh: {}'.format(rel_error(dWh_num, dWh)))
print('Error in dWe: {}'.format(rel_error(dWe_num, dWe)))
print('Error in db1: {}'.format(rel_error(db1_num, db1)))
print('Error in dh_prev: {}'.format(rel_error(dh_prev_num, dh_prev)))

Error in dWh: 3.699756155582621e-11
Error in dWe: 2.8456523039254035e-10
Error in db1: 2.1963488162818666e-11
Error in dh_prev: 2.3951187440793865e-11


In [12]:
""" Check Gradients for the entire RNN """

np.random.seed(10151)
T, N, Dh, d, V = 10, 2, 5, 3, 50
# Parameters
Wh = np.random.randn(Dh, Dh)
b1 = np.random.randn(Dh)
We = np.random.randn(Dh, d)

# Inputs
x = np.random.randn(N, T, d)
h_0 = np.random.randn(N, Dh)

# Test functions
fWh = lambda Wh: rnn_forward(T, x, h_0, Wh, We, b1)[0]
fWe = lambda We: rnn_forward(T, x, h_0, Wh, We, b1)[0]
fb1 = lambda b1: rnn_forward(T, x, h_0, Wh, We, b1)[0]

# Evaluate test functions
h, cache_dict = rnn_forward(T, x, h_0, Wh, We, b1)
dh = np.random.randn(*h.shape)
dWh, dWe, db1 = rnn_backward(dh, cache_dict, Wh, We, b1)
dWh_num = eval_grad(fWh, Wh, dh)
dWe_num = eval_grad(fWe, We, dh)
db1_num = eval_grad(fb1, b1, dh)
print('Error in dWh: {}'.format(rel_error(dWh_num, dWh)))
print('Error in dWe: {}'.format(rel_error(dWe_num, dWe)))
print('Error in db1: {}'.format(rel_error(db1_num, db1)))

Error in dWh: 9.831792282221475e-10
Error in dWe: 1.0017182281175608e-10
Error in db1: 9.769693584166271e-11


In [13]:
""" Check Gradients for the Affine layer """

np.random.seed(10151)
N, T, Dh, V = 5, 10, 6, 50

# Parameters
U = np.random.randn(V, Dh)
b2 = np.random.rand(V)

# Inputs
h = np.random.randn(N, T, Dh)

# Test Functions
fU = lambda U: affine_forward(h, U, b2)[0]
fb2 = lambda b2: affine_forward(h, U, b2)[0]
fh = lambda h: affine_forward(h, U, b2)[0]

# Evaluate test functions
theta, cache = affine_forward(h, U, b2)
dtheta = np.random.randn(*theta.shape)
dh, dU, db2 = affine_backward(dtheta, cache)

dU_num = eval_grad(fU, U, dtheta)
db2_num = eval_grad(fb2, b2, dtheta)
dh_num = eval_grad(fh, h, dtheta)

print('Error in dU: {}'.format(rel_error(dU_num, dU)))
print('Error in db2: {}'.format(rel_error(db2_num, db2)))
print('Error in dh: {}'.format(rel_error(dh_num, dh)))

Error in dU: 7.956108235981939e-10
Error in db2: 1.578675630521908e-09
Error in dh: 1.0117441876769132e-09


<a id='sec_id'></a>
### Training our RNN

In [216]:
N = 64
T = 50
Dh = 64

train_file = 'Berkshire Letters.txt'
with open(train_file, 'r') as f:
    data = f.read()
chars = list(set(data))
data_size, V = len(data), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# Parameters initialization
Wh = np.random.randn(Dh, Dh)
b1 = np.zeros(Dh)
We = np.random.randn(Dh, V)
U = np.random.randn(V, Dh)
b2 = np.zeros(V)

# Mem for Adagrad
mWh, mWe, mU = np.zeros_like(Wh), np.zeros_like(We), np.zeros_like(U)
mb1, mb2 = np.zeros_like(b1), np.zeros_like(b2)

# Other variables' initialization
h_0 = np.zeros((N, Dh))

def str_to_idx(st):
    idx_arr = np.array([char_to_ix[ch] for ch in st])
    return idx_arr
lr = 0.01
loss_freq = 100
num_iter = 0
num_epochs = 15
for epoch in range(num_epochs):
    start = 0
    iter_count = 0
    running_loss = 0
    while True:
        num_iter += 1
        iter_count += 1
        batch_str = [data[(start + i*T):(start + (i+1)*T) + 1] for i in range(N)]
        batch_idx = [str_to_idx(st) for st in batch_str if len(st) == (T+1)]
        batch_size = len(batch_idx)
        if batch_size < N:
            batch_idx.extend([(T+1)*[0] for i in range(N - batch_size)])
        x = np.array([np.eye(V)[indices[0:len(indices)-1]] for indices in batch_idx])
        y = np.array([indices[1:] for indices in batch_idx])
        mask = np.ones((N, T))
        mask[batch_size:,:] = 0
        
        # forward pass
        h, cache_dict = rnn_forward(x.shape[1], x, h_0, Wh, We, b1)
        theta, cache = affine_forward(h, U, b2)
        loss, dtheta = softmax(theta, y, mask)
        running_loss += loss
        if iter_count % loss_freq == 0:
            print('Epoch {} Iteration {}, Running Loss {:.2f}'.format(epoch + 1, iter_count, \
                                                                      running_loss/loss_freq/N))
            running_loss = 0
        # backprop
        dh, dU, db2 = affine_backward(dtheta, cache)
        for dz in [dh, dU, db2]: np.clip(dz, -5, 5, out=dz)
        dWh, dWe, db1 = rnn_backward(dh, cache_dict, Wh, We, b1)
        for dz in [dWh, dWe, db1]: np.clip(dz, -5, 5, out=dz)
        
        # update grads
        for z, dz, m in zip([Wh, We, U, b1, b2], 
                            [dWh, dWe, dU, db1, db2],
                            [mWh, mWe, mU, mb1, mb2]):
            m += dz*dz
            z += -lr*dz / np.sqrt(m + 1e-8)
        start += N*T
        if start >= data_size: break

Epoch 1 Iteration 100, Running Loss 2.68
Epoch 1 Iteration 200, Running Loss 1.63
Epoch 1 Iteration 300, Running Loss 1.46
Epoch 1 Iteration 400, Running Loss 1.39
Epoch 1 Iteration 500, Running Loss 1.34
Epoch 1 Iteration 600, Running Loss 1.31
Epoch 1 Iteration 700, Running Loss 1.28
Epoch 2 Iteration 100, Running Loss 2.47
Epoch 2 Iteration 200, Running Loss 1.26
Epoch 2 Iteration 300, Running Loss 1.24
Epoch 2 Iteration 400, Running Loss 1.21
Epoch 2 Iteration 500, Running Loss 1.20
Epoch 2 Iteration 600, Running Loss 1.19
Epoch 2 Iteration 700, Running Loss 1.18
Epoch 3 Iteration 100, Running Loss 2.29
Epoch 3 Iteration 200, Running Loss 1.17
Epoch 3 Iteration 300, Running Loss 1.17
Epoch 3 Iteration 400, Running Loss 1.15
Epoch 3 Iteration 500, Running Loss 1.14
Epoch 3 Iteration 600, Running Loss 1.14
Epoch 3 Iteration 700, Running Loss 1.12
Epoch 4 Iteration 100, Running Loss 2.19
Epoch 4 Iteration 200, Running Loss 1.13
Epoch 4 Iteration 300, Running Loss 1.13
Epoch 4 Iteratio

In [225]:
# sample text
def sampleText(length=100, seed_ch='T'):
    x_t = np.eye(V)[char_to_ix[seed_ch]]
    h_prev = np.zeros(Dh)
    str_out = seed_ch
    for t in range(length):
        h_next = sigmoid(np.matmul(Wh, h_prev) + np.matmul(We, x_t) + b1)
        theta = np.matmul(U, h_next) + b2
        p = np.exp(theta)/np.sum(np.exp(theta))
        idx = np.random.choice(range(V), p=p.ravel())
        h_prev = h_next
        x_t = np.eye(V)[idx]
        str_out += ix_to_char[idx]
    print(str_out)

In [229]:
sampleText(1000)

T e r e l o s   - u 0 d   c e d t e s   e p t h e   c o r i l e   h i b e   t h e c e   o m i l g   A s e   h i n   t h a r   w i n 4 u r r i t U v i n t e e d   f   m e q l e r s d y   s h e   w o r o c e   d   y f e   a s c s a o w h i i l u i l e y . s   b B e m o w   o r ,   n o .   8 H   a n .   . s .   .   .   .   .   .   .   T h e r   o e q u e n t e   t o o 
 ,   f   t e   o   a r a n g   b e i t e   s I 2   I u g 7 7 	   ,   P o s   2   a n v e s e p t r i c i n e e d   .   	 t   .   .   ,   t h e r   a b n t k a n   3 n   o n   g y   m e w o y a 6 d   . W .   .   .   .   O l   n o u s i o t   N i p C   e n   a n k e v t o r   . 9   i n g   h o c   u r a t   ;   h e   	 m T h o n N e   c o n   e n   i t   t o   a r o m e 	 	 
 
 B a s a g c   a l e l o m a c e   I g   d u s x a l l - d i n e a r s k e   y f a r   t h e     t i c i r i t h e u l r   u c t o a n 
 
 
 
 
 
 
 O r   a n   t o   p o r i b e   c a p i l l i b e a m e   n d a r   y a s l t 8 t h a d   c i r e   c a 

In [203]:
d = np.zeros(5)
d.shape

(5,)